# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_sessions_single_user"
FEATURE_SETS  = [
    ["session_duration", "hour"],
    ["session_duration", "weekday"],
    ["session_duration", "hour", "weekday"],
    ["session_duration", "hour_sin", "hour_cos"],
    ["session_duration", "weekday_sin", "weekday_cos"],
    ["session_duration", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier_on_single_users(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Group the dataframe by users
    grouped = df.groupby("user", as_index=False)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in grouped:

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])

            # Filter the dataframe according to the features 
            group_filtered = group[features].copy()

            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group_filtered)
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group_filtered)
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)
        user_predictions["scores"].replace(np.nan, 0, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=6):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon_sessions"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 6/1 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 4690.4049   Inference Time: 83.7789   pAUC: 0.7527 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465787.0   FP: 4705.0   FN: 99.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 5020.7474   Inference Time: 87.7416   pAUC: 0.7550 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465786.0   FP: 4706.0   FN: 99.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 4967.3995   Inference Time: 91.7561   pAUC: 0.7572 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465787.0   FP: 4705.0   FN: 99.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 5243.1731   Inference Time: 91.7551   pAUC: 0.7578 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 465787.0   FP: 4705.0   FN: 99.0   TP: 0.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 3744.1709   Inference Time: 75.7253   pAU

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 6/1 - Features: session_duration, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 2964.3821   Inference Time: 70.4973   pAUC: 0.7481 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 465790.0   FP: 4702.0   FN: 96.0   TP: 3.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=a